# Problem 4.7 
# Implemetation of Non-recursive Decision Tree

In [1]:
import numpy as np
import pandas as pd
import numbers

# 1. Entropy

In [2]:
def entropy(target):
    """
    This function computes the entropy, which has one input:
    1. target: the df column of response
    """
    ent = 0
    
    for element in pd.value_counts(target):
        p = element / len(target)
        ent += -p * np.log2(p)
    
    return ent  

# 2. Information Gain¶

In [3]:
def information_gain(target_name, attribute_name, data):
    """
    This function computes the information gain of an attribute, which has three inputs:
    1. target_name: string, the column name of response
    2. attribute_name: string, the column name of the attribute
    3. data: dataframe
    
    The class of the attribute could be numberic or object 
    """
    ent = entropy(data[target_name])
    df_slice = data[[target_name, attribute_name]]
    
    node_info = [] # store the best split information
    
    # information gain for categorical attributes
    if not np.issubdtype(data[attribute_name].dtype, np.number):
        new_ent = 0
        for attr in pd.value_counts(data[attribute_name]).index:
            dv = df_slice[df_slice[attribute_name] == attr] # the subset of specific attribute value
            
            ent_dv = entropy(dv[target_name]) # the entropy of the subset
            new_ent += len(dv)/len(data) * ent_dv # compute the sum of entropy 
            node_info.append(attr)
            
    # information gain for numerical attributes
    if np.issubdtype(data[attribute_name].dtype, np.number): 
        new_ent = np.inf
        sorted_attr = sorted(data[attribute_name].values)
        points  = [(sorted_attr[i]+sorted_attr[i+1])/2 for i in range(len(sorted_attr)-1)]
        for point in points:
            # compute the entropy for two subsets, + and -
            dv1 = df_slice[df_slice[attribute_name] < point]
            ent_dv1 = len(dv1)/len(data) * entropy(dv1[target_name])
            
            dv2 = df_slice[df_slice[attribute_name] > point]
            ent_dv2 = len(dv2)/len(data) * entropy(dv2[target_name])
            
            # find the smallest entropy sum
            if ent_dv1+ent_dv2 < new_ent:
                new_ent = ent_dv1 + ent_dv2
                node_info = [point]

        
    return ent - new_ent, node_info
    

# 3. Build Tree

In [115]:
def best_split(data, target_name):
    """
    This function returns the best split information (tree stump) of certain dataframe
    """
    
    attributes = list(data.columns) # get all the attributes
    attributes.remove(target_name)
    origin_gain = 0
    
    for attr in attributes:
        gain, node_info = information_gain(target_name, attr, data)
        if gain > origin_gain:
            origin_gain = gain
            node_column_name = attr
            split_info = node_info
    return node_column_name, split_info

In [5]:
def majorClass(data, target_name):
    """
    Majority Function simply tells which class has more entries in given data-set
    """
    value_cnt = pd.value_counts(data[target_name])
    # np.unique(data['quality'])[np.argmax(np.unique(data['quality'],return_counts=True)[1])]
    major = list(value_cnt.index[value_cnt.values == value_cnt.max()])

    return major[0]

In [6]:
def partition_data(data, best_attr_name, value, isnumber = True, islarger = None):
    """
    This function return the new dataframe based on the best split information.
    It has five inputs:
    1. data: the dataframe that should be sliced
    2. best_attr_name: string, the name of the attribute which is going to be splitted on
    3. value: list, the best split values
    4. isnumber: boolean, identify if the best split value is numeric
    5. islarger: boolean, identify if the condition of splitting is 
                 'larger than the best split value'
    """
    
    if isnumber:
        if islarger:
            new_df = data[data[best_attr_name] > value]
        else: new_df = data[data[best_attr_name] < value]
    else: new_df = data[data[best_attr_name] == value]
    return new_df
    

In [42]:
def sub_data_dict(data, best_attr_name, best_split_info):
    """
    return the list of all sub-datas based on a column
    """
    sub_data_dict = {}
    
    if isinstance(best_split_info[0], numbers.Number):
        values = ['<'+str(best_split_info[0]), '>'+str(best_split_info[0])]
        islargers = [False, True]
        for i in range(2):
            value = values[i]
            sub_data = partition_data(data, best_attr_name, best_split_info[0], True, islargers[i])
            sub_data_dict[value] = sub_data
                
                
    else:
        for value in best_split_info:
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = partition_data(data, best_attr_name, value, False)
            sub_data_dict[value] = sub_data
    return sub_data_dict   
    
    

In [246]:
def non_recursive_tree(data,max_depth, target_name='quality'):
    """
    This function builds a decision tree with stack instead of a recursive loop.
    There are three inputs:
    1. data: the df of the train data
    2. max_depth: the max_depth of decision tree
    3. target_name: string, the column name of the response
    """
    
    init_col, init_split = best_split(data, target_name)
    
    root = {init_col:{}}
    data_stack = []
    data_stack.append(data)
    
    col_stack = []
    col_stack.append(init_col)
    
    split_info_stack = []
    split_info_stack.append(init_split)
    
    node_stack = []
    node_stack.append(root[init_col])
    
 
    level_dict = {init_col:1}
    
    
    while (len(data_stack) > 0):
       
        data_partition = data_stack.pop()
        col = col_stack.pop()
        best_split_info = split_info_stack.pop()
        current_point = node_stack.pop()
        
        if level_dict[col] > max_depth:
            current_point = majorClass(data_partition, target_name)
            
        else:
            part_data_dict = sub_data_dict(data_partition, col, best_split_info)
        
            for key in part_data_dict:
                level_dict[key] = level_dict[col]+0.1

            for key in part_data_dict:

                sub_data = part_data_dict[key]

                if len(pd.value_counts(sub_data[target_name])) == 1:

                    end_label = majorClass(sub_data,target_name)
                    label = col

                    if label in current_point.keys():
                        current_point[label][key] = end_label

                    else: 
                        current_point[key] = end_label
                    
                    continue

                elif len(pd.value_counts(sub_data[target_name])) > 1:
                    
                    if level_dict[key] == max_depth+0.1:
                        current_point[key] = majorClass(sub_data, target_name)
                        
                    elif level_dict[key] < max_depth:
                        col, split = best_split(sub_data, target_name)
                        current_point[key] = {}
                        current_point[key][col] = {}

                        data_stack.append(sub_data)
                        col_stack.append(col)
                        node_stack.append(current_point[key])
                        split_info_stack.append(split)

                        level_dict[col] = level_dict[key]+1-0.1

                    continue

    return root
    

# 4. Predict

In [238]:
def check_float(value):
    """
    check if a string input is numeric
    """
    try:
        f = float(value)
        return True
    except: return False
    

In [239]:
def predict(input_df,tree,default = 'No prediction'):
    """
    Prediction of a new/unseen dataframe instance. This takes two parameters:
    1. input_df: a row of new instance with column names
    2. tree: the built decision tree
    3. default value: return the value in case of 
       new/unseen instance contains unseen attribute value.
    
    Also it is made in a recrusive manner.
    """
    for column in input_df:
        if column in list(tree.keys()):
            
            attr_value = list(input_df[column])[0]
            
            if isinstance(attr_value, numbers.Number):
                threshold = float(list(tree[column].keys())[0][1:])
                if attr_value < threshold:
                    attr_value = '<'+str(threshold)
                else: 
                    attr_value = '>'+str(threshold)
                    result = tree[column][attr_value]
            else: # do categorical classification
                try:

                    result = tree[column][attr_value] 
                except:
                    return default
  
            result = tree[column][attr_value]

            if isinstance(result,dict):
                return predict(input_df,result)
            else:
                return result

# 5. Train the Tree

In [228]:
data = pd.read_csv('../data/data.txt', sep=',')
data = data.drop(['Id'],axis=1)

In [240]:
data.head()

,color,root,sound,stripes,umbilical,touch,density,sugar,quality
0,dark-green,roll-up,dull,clear,hollow,hard,0.697,0.460,good
1,pitch-dark,roll-up,dead,clear,hollow,hard,0.744,0.376,good
2,pitch-dark,roll-up,dull,clear,hollow,hard,0.634,0.264,good
3,dark-green,roll-up,dead,clear,hollow,hard,0.608,0.318,good
4,white,roll-up,dull,clear,hollow,hard,0.556,0.215,good


In [241]:
# split the train and test set
train=data.sample(frac=0.9,random_state=4)

# the test set should only contain features values
test_x, test_y =data.drop(train.index).drop(['quality'], axis=1), data.drop(train.index)['quality']

In [247]:
# fit the tree
tree0 = non_recursive_tree(train, 3)

In [248]:
tree0

{'stripes': {'blurred': 'bad',
  'clear': {'root': {'roll-up': 'good',
    'slighly-curled': 'good',
    'stiff': 'bad'}},
  'indistinct': {'touch': {'hard': 'bad', 'soft': 'good'}}}}

In [251]:
tree1 = non_recursive_tree(train, 1)
tree1

{'stripes': {'blurred': 'bad', 'clear': 'good', 'indistinct': 'bad'}}

In [253]:
# make prediction
prediction_result = []

for i in range(len(test_x)):
    new_pred = predict(test_x.iloc[[i]], tree0)
    prediction_result.append(new_pred)

In [254]:
prediction_result

['good', 'good']

In [255]:
test_y

7     good
14     bad
Name: quality, dtype: object